### 3W compression notebook

This notebook transforms the 3W dataset into a compressed version. The time series is down-sampled from entries everry second to ten seconds, by a locaal averrageg over this period. Aditionally, features P-CKGL and T-CKGL are removed from the dataset since they only appear in real instances.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
data_path = Path('data')

In [10]:
def class_and_file_generator(data_path, real=False, simulated=False, drawn=False):
    for class_path in data_path.iterdir():
        if class_path.is_dir():
            class_code = int(class_path.stem)
            for instance_path in class_path.iterdir():
                if (instance_path.suffix == '.csv'):
                    if (simulated and instance_path.stem.startswith('SIMULATED')) or \
                       (drawn and instance_path.stem.startswith('DRAWN')) or \
                       (real and (not instance_path.stem.startswith('SIMULATED')) and \
                       (not instance_path.stem.startswith('DRAWN'))):
                        yield class_code, instance_path

In [96]:
real_instances = list(class_and_file_generator(data_path, real=True, simulated=False, drawn=False))
simulated_instances = list(class_and_file_generator(data_path, real=False, simulated=True, drawn=False))
drawn_instances = list(class_and_file_generator(data_path, real=False, simulated=False, drawn=True))

In [102]:
instances = real_instances + simulated_instances

In [103]:
import os

def create_dir_tree(root_dir):
    os.mkdir(root_dir)
    for i in range(0, 9):
        path = os.path.join(root_dir, str(i))
        os.mkdir(path)


data_root_dir = "data_transformed"

create_dir_tree(data_root_dir)

for c, p in instances:
    if c == 0:
        df = pd.read_csv(p)
        df['timestamp'] = pd.to_datetime(df['timestamp'])  #down-sampling
        df = df.resample('10s', on='timestamp').mean()
        df.drop('P-JUS-CKGL', inplace=True, axis=1)        #removing P-CKGL and T-CKGL attributes
        df.drop('T-JUS-CKGL', inplace=True, axis=1)
        path = os.path.join(data_root_dir, str(p)[5:])
        df.to_csv(path)

1025

1984